In [2]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd

url = "https://es.uefa.com/uefachampionsleague/history/seasons/1993/statistics/players/goals/?sortBy=assists&order=asc"

session = requests.Session()

# Reintentos ante errores de conexión, 5xx, etc.
retries = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["HEAD", "GET", "OPTIONS"]
)
session.mount("https://", HTTPAdapter(max_retries=retries))
session.mount("http://", HTTPAdapter(max_retries=retries))

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/129.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "es-ES,es;q=0.9,en;q=0.8",
}

try:
    resp = session.get(url, headers=headers, timeout=15)
    resp.raise_for_status()
    print("Status:", resp.status_code)
    print("Primeros 500 chars de HTML:")
    print(resp.text[:500])

    tables = pd.read_html(resp.text)
    df = tables[0]
    df.to_csv("ucl_1993_assists.csv", index=False)
    print("CSV creado OK")

except Exception as e:
    print("Ha fallado la descarga:", repr(e))



Ha fallado la descarga: ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'es.uefa.com\', port=443): Max retries exceeded with url: /uefachampionsleague/history/seasons/1993/statistics/players/goals/?sortBy=assists&order=asc (Caused by ReadTimeoutError("HTTPSConnectionPool(host=\'es.uefa.com\', port=443): Read timed out. (read timeout=15)"))'))
